# Visualization of CAR map
## For model 2
Results calculated in R
Visualizing in Python
This is a final version

In [ ]:
%matplotlib inline
import sys
sys.path.append('/apps')
import django
django.setup()
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
## Use the ggplot style
plt.style.use('ggplot')

In [ ]:
#file = '/outputs/presence_only_models/predictors/dataset100x100-puebla-p9/0-pred.csv'
#PDF = read.csv(file)
## REad adjancency matrix
mat_filename = "/outputs/training_data_sample_puebla_p9_abies_pinophyta_adjmat.npy"


#TDF = pd.read_csv("/outputs/training_data_sample_puebla_p9_abies_pinophyta.csv")
TDF = pd.read_csv("/outputs/training_data_sample_puebla_p9_tyrannidae_birds.csv")
## Order it according to the id of the cell
## This is important because the adjancy matrix rows need to be the same

TDF.sort_values(by='cell_ids',inplace=True)

plt.plot(TDF.cell_ids.values)

#gTDF = gpd.GeoDataFrame(TDF,geometry='geometry')


In [ ]:
import utilities.data_extraction as de
nTDF = de.toGeoDataFrame(TDF,xcoord_name='Longitude',ycoord_name='Latitude')

## Extract the polygons from cells


## Run this to load geometries directly from saved file
### it's faster,[Required to make this efficient]

In [ ]:
## Geometry file
geometryfile = "/outputs/training_data_sample_puebla_p9_geometry.json"
## Readfile
gg = gpd.read_file(geometryfile)
gg.set_index('indx',inplace=True)
plt.rcParams['figure.figsize'] = [10, 10]
gg.plot(edgecolor='black')

### Develop here new pseudo-absences strategies designs
This has been done in R. 
Let's see how we can use it from here

> IMPOSIBLE TO INSTALL new version of Rpy2 due to stupid python 2/7
Continue evading the problem, as the water....

In [ ]:
# Read from csv
#file_ = '/outputs/presence_only_models/modelCAR2bis.csv'
#file_ = '/outputs/presence_only_models/modelCAR2Prec.csv'
file_ = '/outputs/presence_only_models/modelCAR2_tyrannidae.csv'
data = pd.read_csv(file_)
data = data.drop(data.columns[0],axis=1)
data.set_index(data.columns[0],inplace=True)
data.set_index(data.index.astype('int'),inplace=True)
#fitted_sample = fitted_sample.transpose()
## ## Join it with geodataframe
gdf = gg.join(data,how='inner')
gdf.crs = {'init' : 'epsg:4326'}

### Plot with GeoViews

In [ ]:
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs
gv.extension('bokeh','matplotlib')

### Plot the Common latent GMRF 

In [ ]:
tiles = gv.tile_sources.EsriImagery
labels = gv.tile_sources.EsriReference 


sample_pt = nTDF[nTDF.Aves > 0]
#pines_pt = gTDF[gTDF.Abies > 0]
pines_pt = nTDF[nTDF.Tyrannidae > 0]

width = 600
height = 650

group_name = 'Model 2 - Common Latent GMRF'
lq2 = 'Quantile: 0.025'
lq9 = 'Quantile: 0.975'
lm = 'Mean:'

cmap = plt.cm.viridis
sample_pt = gv.Points((sample_pt.Longitude,sample_pt.Latitude),group=group_name,label='Sample').opts(
            fill_color = 'red',
            line_color = 'grey',
            line_width = 0.2,
            line_alpha = 0.9,
            fill_alpha = 0.6,
            size = 2,
            )

presence_pt = gv.Points((pines_pt.Longitude,pines_pt.Latitude),group=group_name,label='Presence').opts(
           marker = 'x',
          color = 'black')

obs = (sample_pt * presence_pt)



gq2 = gv.Polygons(gdf,vdims=['phi_q025'],group=group_name,label=lq2).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,
    cmap = cmap
)


gm = gv.Polygons(gdf,vdims=['phi_mean'],group=group_name,label=lm).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,   
    cmap = cmap
)

gq9 = gv.Polygons(gdf,vdims=['phi_q0975'],group=group_name,label=lq9).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,   
    cmap = cmap
)

latent_pres_fig = ((gq2* obs).relabel(lq2) + (gm * obs).relabel(lm) + (gq9 * obs).relabel(lq9)).opts(merge_tools=False)

In [ ]:
latent_pres_fig


In [ ]:
gdf.columns

## Plot the Common Latent GMRF for 'nugget'

In [ ]:
tiles = gv.tile_sources.EsriImagery
labels = gv.tile_sources.EsriReference 


sample_pt = nTDF[nTDF.Aves > 0]
#pines_pt = gTDF[gTDF.Abies > 0]
pines_pt = nTDF[nTDF.Tyrannidae > 0]
width = 600
height = 650

cmap = plt.cm.viridis

group_name = 'Model 2 - Common GMRF with linear random effect (sample)'
lq2 = 'Quantile: 0.025'
lq9 = 'Quantile: 0.975'
lm = 'Mean:'
sample_pt = gv.Points((sample_pt.Longitude,sample_pt.Latitude),group=group_name,label='Sample').opts(
            fill_color = 'red',
            line_color = 'grey',
            line_width = 0.2,
            line_alpha = 0.9,
            fill_alpha = 0.6,
            size = 2,
            )

presence_pt = gv.Points((pines_pt.Longitude,pines_pt.Latitude),group=group_name,label='Presence').opts(
           marker = 'x',
          color = 'black')

obs = (sample_pt * presence_pt)



gq2 = gv.Polygons(gdf,vdims=['reS_q025'],group=group_name,label=lq2).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,
    cmap = cmap
)


gm = gv.Polygons(gdf,vdims=['reS_mean'],group=group_name,label=lm).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,   
    cmap = cmap
)

gq9 = gv.Polygons(gdf,vdims=['reS_q0975'],group=group_name,label=lq9).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,   
    cmap = cmap
)

latent_sample_fig = ((gq2* obs).relabel(lq2) + (gm * obs).relabel(lm) + (gq9 * obs).relabel(lq9)).opts(merge_tools=False)

In [ ]:
latent_sample_fig

In [ ]:
#### RE. P

tiles = gv.tile_sources.EsriImagery
labels = gv.tile_sources.EsriReference 


sample_pt = nTDF[nTDF.Aves > 0]
#pines_pt = gTDF[gTDF.Abies > 0]
pines_pt = nTDF[nTDF.Tyrannidae > 0]


group_name = 'Model 2 - Common GMRF with linear random effect (presence)'
lq2 = 'Quantile: 0.025'
lq9 = 'Quantile: 0.975'
lm = 'Mean:'
sample_pt = gv.Points((sample_pt.Longitude,sample_pt.Latitude),group=group_name,label='Sample').opts(
            fill_color = 'red',
            line_color = 'grey',
            line_width = 0.2,
            line_alpha = 0.9,
            fill_alpha = 0.6,
            size = 2,
            )

presence_pt = gv.Points((pines_pt.Longitude,pines_pt.Latitude),group=group_name,label='Presence').opts(
           marker = 'x',
          color = 'black')

obs = (sample_pt * presence_pt)



gq2 = gv.Polygons(gdf,vdims=['reP_q025'],group=group_name,label=lq2).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,
    cmap = cmap
)


gm = gv.Polygons(gdf,vdims=['reP_mean'],group=group_name,label=lm).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,   
    cmap = cmap
)

gq9 = gv.Polygons(gdf,vdims=['reP_q0975'],group=group_name,label=lq9).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,   
    cmap = cmap
)

latent_sample_fig = ((gq2* obs).relabel(lq2) + (gm * obs).relabel(lm) + (gq9 * obs).relabel(lq9)).opts(merge_tools=False)

In [ ]:
latent_sample_fig

## Joint probability of Presence and Sample

In [ ]:
tiles = gv.tile_sources.EsriImagery
labels = gv.tile_sources.EsriReference 

sample_pt = nTDF[nTDF.Aves > 0]
#pines_pt = gTDF[gTDF.Abies > 0]
pines_pt = nTDF[nTDF.Tyrannidae > 0]


group_name = 'Model 2 - Joint probability of Presence and Sample'
lq2 = 'Quantile: 0.025'
lq9 = 'Quantile: 0.975'
lm = 'Mean:'

cmap = plt.cm.gnuplot2
clim = (0,1)

sample_pt = gv.Points((sample_pt.Longitude,sample_pt.Latitude),group=group_name,label='Sample').opts(
            fill_color = 'white',
            line_color = 'black',
            marker = '^',
            line_width = 0.4,
            line_alpha = 0.9,
            fill_alpha = 0.6,
            size = 2,
            )

presence_pt = gv.Points((pines_pt.Longitude,pines_pt.Latitude),group=group_name,label='Presence').opts(
           marker = 'x',
          color = 'black')

obs = (sample_pt * presence_pt)



gq2 = gv.Polygons(gdf,vdims=['PS_q025'],group=group_name,label=lq2).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    clim = clim,
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,
    cmap = cmap
)


gm = gv.Polygons(gdf,vdims=['PS_mean'],group=group_name,label=lm).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,
    clim = clim,
    cmap = cmap
)

gq9 = gv.Polygons(gdf,vdims=['PS_q0975'],group=group_name,label=lq9).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,
    clim = clim,
    cmap = cmap
)

latent_p_fig = ((gq2* obs).relabel(lq2) + (gm * obs).relabel(lm) + (gq9 * obs).relabel(lq9)).opts(merge_tools=False)

In [ ]:
latent_p_fig

### Plot the Latent variable P (presence)

In [ ]:
tiles = gv.tile_sources.EsriImagery
labels = gv.tile_sources.EsriReference 


sample_pt = nTDF[nTDF.Aves > 0]
#pines_pt = gTDF[gTDF.Abies > 0]
pines_pt = nTDF[nTDF.Tyrannidae > 0]


group_name = 'Model 2 - Probability of latent presence'
lq2 = 'Quantile: 0.025'
lq9 = 'Quantile: 0.975'
lm = 'Mean:'

cmap = plt.cm.gnuplot2
clim = (0,1)

width = 600
height = 650

sample_pt = gv.Points((sample_pt.Longitude,sample_pt.Latitude),group=group_name,label='Sample').opts(
            fill_color = 'white',
            line_color = 'black',
            marker = '^',
            line_width = 0.4,
            line_alpha = 0.9,
            fill_alpha = 0.6,
            size = 2,
            )

presence_pt = gv.Points((pines_pt.Longitude,pines_pt.Latitude),group=group_name,label='Presence').opts(
           marker = 'x',
          color = 'black')

obs = (sample_pt * presence_pt)



gq2 = gv.Polygons(gdf,vdims=['P_q025'],group=group_name,label=lq2).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    clim = clim,
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,
    cmap = cmap
)


gm = gv.Polygons(gdf,vdims=['P_mean'],group=group_name,label=lm).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,
    clim = clim,
    cmap = cmap
)

gq9 = gv.Polygons(gdf,vdims=['P_q0975'],group=group_name,label=lq9).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,
    clim = clim,
    cmap = cmap
)

latent_p_fig = ((gq2* obs).relabel(lq2) + (gm * obs).relabel(lm) + (gq9 * obs).relabel(lq9)).opts(merge_tools=False)

In [ ]:
latent_p_fig

### Plot the latent Sample S

In [ ]:
tiles = gv.tile_sources.EsriImagery
labels = gv.tile_sources.EsriReference 


sample_pt = nTDF[nTDF.Aves > 0]
#pines_pt = gTDF[gTDF.Abies > 0]
pines_pt = nTDF[nTDF.Tyrannidae > 0]

width = 600
height = 650

cmap = plt.cm.gnuplot2
clim = (0,1)

group_name = 'Model 2 - Probability of latent sample'
lq2 = 'Quantile: 0.025'
lq9 = 'Quantile: 0.975'
lm = 'Mean:'
sample_pt = gv.Points((sample_pt.Longitude,sample_pt.Latitude),group=group_name,label='Sample').opts(
            fill_color = 'white',
            line_color = 'black',
            marker = '^',
            line_width = 0.4,
            line_alpha = 0.9,
            fill_alpha = 0.6,
            size = 2,
            )

presence_pt = gv.Points((pines_pt.Longitude,pines_pt.Latitude),group=group_name,label='Presence').opts(
           marker = 'x',
          color = 'black')

obs = (sample_pt * presence_pt)



gq2 = gv.Polygons(gdf,vdims=['S_q025'],group=group_name,label=lq2).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    clim = clim,    
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,
    cmap = cmap
)


gm = gv.Polygons(gdf,vdims=['S_mean'],group=group_name,label=lm).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    clim = clim,
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,   
    cmap = cmap
)

gq9 = gv.Polygons(gdf,vdims=['S_q0975'],group=group_name,label=lq9).opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    clim = clim,
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,   
    cmap = cmap
)

latent_s_fig = ((gq2* obs).relabel(lq2) + (gm * obs).relabel(lm) + (gq9 * obs).relabel(lq9)).opts(merge_tools=False)

In [ ]:
latent_s_fig

## Plot Probability of Sample, Presence and joint

In [ ]:
tiles = gv.tile_sources.EsriImagery
labels = gv.tile_sources.EsriReference 


sample_pt = nTDF[nTDF.Aves > 0]
#pines_pt = gTDF[gTDF.Abies > 0]
pines_pt = nTDF[nTDF.Tyrannidae > 0]

width = 600
height = 650

group_name = '- Model 2'
lq2 = 'Quantile: 0.025'
lq9 = 'Quantile: 0.975'
lm = 'Mean:'

cmap = plt.cm.RdYlBu
clim = (0,1)


sample_pt = gv.Points((sample_pt.Longitude,sample_pt.Latitude),group=group_name,label='Sample').opts(
            fill_color = 'white',
            line_color = 'black',
            marker = '^',
            line_width = 0.4,
            line_alpha = 0.9,
            fill_alpha = 0.6,
            size = 2,
            )

presence_pt = gv.Points((pines_pt.Longitude,pines_pt.Latitude),group=group_name,label='Presence').opts(
           marker = 'x',
          color = 'black')

obs = (sample_pt * presence_pt)



px = gv.Polygons(gdf,vdims=['ProbS'],group=group_name,label='X').opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    clim = clim,
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,
    cmap = cmap
)


py = gv.Polygons(gdf,vdims=['ProbP'],group=group_name,label='Y').opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    clim = clim,
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,   
    cmap = cmap
)

pxy = gv.Polygons(gdf,vdims=['ProbPS'],group=group_name,label='Joint probability P,S').opts(
    tools=['hover'],
    width = width,
    height = height,
    colorbar = True,
    colorbar_position = 'bottom',
    clim = clim,
    alpha = 1.0,
    line_width = 0.001,
    line_alpha = 0.2,   
    cmap = cmap
)
group_name = 'Probability of '
latent_xy_fig = ((px* obs).relabel(group_name + 'Sample') + (py * obs).relabel(group_name + 'Presence') + (pxy * obs).relabel('Joint ' + group_name + 'P,S')).opts(merge_tools=False)

In [ ]:
latent_xy_fig